# Escherichia coli and Saccharomices cerevisiae co-culture

The notebook illustrates how to 
- construct a community model representing the co-culture of Escherichia coli and Saccharomices cerevisiae from models of each single organism,
- run FBA on the community model
- optimize the co-culture for the production of a naringenin.

In [1]:
from cobra.io import read_sbml_model

from mewpy.optimization import EA
from mewpy.optimization.evaluation import TargetFlux, BPCY
from mewpy.problems import RKOProblem
from mewpy import get_simulator
from mewpy.com import *

## Load individual organism model

In [2]:
sc = read_sbml_model('models/yeast/iMM904.xml.gz')
ec = read_sbml_model('models/ec/iAF1260.xml')
get_simulator(ec).objective='BIOMASS'

Set parameter Username
Academic license - for non-commercial use only - expires 2024-12-11


No objective coefficients in model. Unclear what should be optimized


# Community Model and Medium

In [3]:
community = CommunityModel([ec,sc])
cmodel = community.merged_model

Organism:   0%|                                                                      | 0/2 [00:00<?, ?it/s]Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.
Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.
Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.
Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.

In [4]:
for rx in cmodel.get_exchange_reactions():
    cmodel.set_reaction_bounds(rx,0,1000,False)

In [5]:
medium = dict()

minimal_medium = ['R_EX_xyl__D_e', 'R_EX_ac_e_iMM904', 'R_EX_mobd_e', 'R_EX_cl_e', 
                  'R_EX_cobalt2_e','R_EX_mn2_e', 'R_EX_mg2_e', 'R_EX_ca2_e', 'R_EX_k_e', 
                  'R_EX_zn2_e', 'R_EX_xyl__D_e_iAF1260', 'R_EX_so4_e', 
                  'R_EX_cu_e', 'R_EX_fe3_e', 'R_EX_o2_e', 'R_EX_pi_e', 
                  'R_EX_nh4_e']

# Carbon sources are constrained
for exchange in minimal_medium:
    if exchange != 'R_EX_xyl__D_e' and exchange != 'R_EX_ac_e_iMM904':
        medium[exchange]=(-1000, 0)
    else:
        medium[exchange]=(-10, 1000)

# Guarantee that L-Tyrosine is only produced by the organisms
medium['R_EX_tyr__L_e'] = 0

cmodel.set_environmental_conditions(medium)

In [6]:
res = cmodel.simulate(method='pFBA')

In [7]:
res.get_metabolite('M_tyr__L_e')

,Value
Reaction,
R_EX_tyr__L_e_iAF1260,-0.057137
R_EX_tyr__L_e_iMM904,0.057137
R_EX_tyr__L_e,-0.000000


In [8]:
cmodel.FVA(reactions=['R_EX_tyr__L_e_iAF1260','R_EX_tyr__L_e_iMM904'])

{'R_EX_tyr__L_e_iAF1260': [-0.06201572827296663, 0.4629281063033095],
 'R_EX_tyr__L_e_iMM904': [-0.46292810649603566, 0.062015728275737037]}

## Optimization

In [10]:
TARGET = 'R_EX_tyr__L_e_iAF1260'

In [11]:
BIOMASS = list(cmodel.objective.keys())[0]
BIOMASS

'community_growth'

In [12]:
f1 = BPCY(BIOMASS,TARGET)
f2 = TargetFlux(TARGET)

In [13]:
essential = cmodel.essential_reactions()

100%|██████████████████████████████████████████████████████████████████| 4320/4320 [06:58<00:00, 10.32it/s]


In [14]:
KO_targets = []

for rxn in cmodel.reactions:
        if rxn.endswith('iAF1260') and rxn not in essential:
            if (rxn == 'R_ATPM_iAF1260'
                or rxn.startswith('R_EX_') 
                or rxn.startswith('R_ATPS')
                or rxn.endswith('tex_iAF1260')
                or rxn.endswith('pp_iAF1260')
                or rxn.endswith('exi_iAF1260')):
                continue
            else:
                KO_targets.append(rxn)

In [15]:
len(KO_targets)

1038

In [16]:
problem = RKOProblem(cmodel, 
                     fevaluation=[f1,f2],
                     target=KO_targets,
                     candidate_max_size=2)

ea = EA(problem, max_generations = 10)

In [17]:
ea.run(simplify=False)

Running NSGAII
Eval(s)|     Worst      Best    Median   Average   Std Dev|     Worst      Best    Median   Average   Std Dev|
    100|  -0.023636  -0.022767  -0.023636  -0.023596  0.000149|  -0.057137  -0.056078  -0.057137  -0.057089  0.000181|
    200|  -0.023636  -0.022767  -0.023636  -0.023579  0.000167|  -0.057137  -0.056078  -0.057137  -0.057068  0.000203|
    300|  -0.023636  -0.022746  -0.023614  -0.023511  0.000222|  -0.057137  -0.056051  -0.057111  -0.056985  0.000271|
    400|  -0.023601  -0.022582  -0.023396  -0.023312  0.000289|  -0.057095  -0.055849  -0.056847  -0.056743  0.000352|
    500|  -0.023167  -0.015257  -0.023053  -0.022875  0.000788|  -0.056568  -0.037559  -0.056428  -0.056117  0.001879|
    600|  -0.022953  -0.015257  -0.022797  -0.022507  0.001291|  -0.056306  -0.037559  -0.056114  -0.055479  0.003162|
    700|  -0.022767  -0.015257  -0.022491  -0.022096  0.001744|  -0.056078  -0.037559  -0.055736  -0.054693  0.004339|
    800|  -0.022400  0.016952  -0.022241 

[[0.016951661844721137, 0.041582081575620475];{'R_CS_iAF1260': 0, 'R_PPM_iAF1260': 0},
 [-0.015257349329576394, -0.03755919954146343];{'R_3OAR80_iAF1260': 0, 'R_PPM_iAF1260': 0},
 [-0.015257349329576394, -0.03755919954146343];{'R_3OAS60_iAF1260': 0, 'R_PPM_iAF1260': 0}]

In [18]:
ea.dataframe()

,Modification,MSize,BPCY,TargetFlux
0,"{'R_CS_iAF1260': 0, 'R_PPM_iAF1260': 0}",2,0.016952,0.041582
1,"{'R_3OAR80_iAF1260': 0, 'R_PPM_iAF1260': 0}",2,-0.015257,-0.037559
2,"{'R_3OAS60_iAF1260': 0, 'R_PPM_iAF1260': 0}",2,-0.015257,-0.037559


# Evaluate solutions

In [19]:
solution = {'R_PYK_iAF1260':0, 'R_PPNDH_iAF1260':0}
res = cmodel.simulate(method='pFBA',constraints=solution)

In [20]:
res.find('tyr')

,Flux rate
Reaction ID,
R_EX_tyr__L_e_iAF1260,0.041595
R_EX_tyr__L_e_iMM904,-0.041595


In [21]:
res.find('BIO')

,Flux rate
Reaction ID,
R_BIOMASS_iAF1260,0.407792
R_BIOMASS_SC5_notrace_iMM904,0.407792


Identify additional interactions:

In [22]:
exchanges(community,res)

,iAF1260,iMM904,Total Net
Metabolite,,,
M_pro__L_e,-0.090163,0.090163,0.00000
M_mobd_e,-0.001288,0.000000,-0.00129
M_cl_e,-0.001932,0.000000,-0.00193
M_asn__L_e,0.041472,-0.041472,0.00000
M_cobalt2_e,-0.001288,0.000000,-0.00129
M_orn_e,-0.186157,0.186157,0.00000
M_h2o_e,22.699018,11.551667,34.25068
M_co2_e,10.161833,8.482286,18.64412
M_fum_e,-14.169094,14.169094,0.00000
